In [1]:
import numpy as np
import pandas as pd

def parse_complete_data():
    """解析图片中的完整数据集"""
    # 从图片中提取的所有数据
    data = [
        [0.4733, 0.3366, 0.1901],
        [0.5313, 0.2972, 0.1715],
        [0.2671, 0.3204, 0.4124]
    ]
    return pd.DataFrame(data, columns=['class0', 'class1', 'class2', 'result'])

def evaluate_predictions(df, threshold_single=0.5, threshold_gap=0.15, threshold_gap_2=0.1, threshold_sum=0.8):
    """评估预测结果"""
    results = []
    correct = 0
    total = len(df)
    single_predictions = 0
    dual_predictions = 0
    
    for idx, row in df.iterrows():
        probs = [row['class0'], row['class1'], row['class2']]
        true_label = row['result']
        
        # 获取排序后的概率和对应的类别索引
        sorted_indices = np.argsort(probs)[::-1]
        sorted_probs = np.array(probs)[sorted_indices]
        
        max_prob = sorted_probs[0]
        second_prob = sorted_probs[1]
        last_prob = sorted_probs[-1]
        prob_gap = max_prob - second_prob
        
        # 确定预测策略
        if (max_prob >= 0.35) and (second_prob >= 0.35) and (max_prob <= 0.4) and (second_prob <= 0.4):
            predictions = [1]
            pred_type = "draw 特殊处理1"
            single_predictions += 1
            print("draw 特殊处理1: ", probs)
        elif (max_prob >= 0.35) and (second_prob >= 0.35) and last_prob < 0.2:
            predictions = [1]
            pred_type = "draw 特殊处理1"
            single_predictions += 1
            print("draw 特殊处理1: ", probs)
        elif probs[1] >=0.35:
            predictions = [1]
            pred_type = "draw 特殊处理2"
            single_predictions += 1
            print("draw 特殊处理2: ", probs)
        elif second_prob >= 0.28 and second_prob <= 0.32 and np.abs(second_prob - last_prob) <= 0.01:
            predictions = [1]
            pred_type = "draw 特殊处理3"
            single_predictions += 1
            print("draw 特殊处理3: ", probs)
        elif max_prob >= 0.49 and max_prob <= 0.51:
            predictions = [1]
            pred_type = "draw 特殊处理4"
            single_predictions += 1
            print("draw 特殊处理4: ", probs)
        elif (max_prob + second_prob) >= threshold_sum:
            predictions = [sorted_indices[0], sorted_indices[1]]
            pred_type = "双重预测"
            dual_predictions += 1
            print("双重预测: ", probs)
        elif max_prob >= threshold_single and prob_gap >= threshold_gap:
            predictions = [sorted_indices[0]]
            pred_type = "单一预测"
            single_predictions += 1
            print("单一预测: ", probs)

        else:
            predictions = [sorted_indices[0]]
            pred_type = "默认单一预测"
            single_predictions += 1
            print("默认单一预测: ", probs)
        
        # 检查预测是否正确
        is_correct = true_label in predictions
        if is_correct:
            correct += 1
            
        results.append({
            'index': idx + 1,
            'true_label': true_label,
            'predictions': predictions,
            'prediction_type': pred_type,
            'max_prob': max_prob,
            'second_prob': second_prob,
            'is_correct': is_correct
        })
    
    accuracy = correct / total
    
    return {
        'detailed_results': results,
        'summary': {
            'total_samples': total,
            'correct_predictions': correct,
            'accuracy': accuracy,
            'single_predictions': single_predictions,
            'dual_predictions': dual_predictions
        }
    }

# 执行分析
df = parse_complete_data()
evaluation = evaluate_predictions(df)

# 打印总体结果
summary = evaluation['summary']
print("\n总体分析结果:")
print(f"总样本数: {summary['total_samples']}")
print(f"正确预测数: {summary['correct_predictions']}")
print(f"最终准确率: {summary['accuracy']:.2%}")
print(f"单一预测数量: {summary['single_predictions']}")
print(f"双重预测数量: {summary['dual_predictions']}")

# 打印详细的预测分布
detailed_results = pd.DataFrame(evaluation['detailed_results'])
prediction_type_stats = detailed_results['prediction_type'].value_counts()
print("\n预测类型分布:")
print(prediction_type_stats)

# 计算每种预测类型的准确率
print("\n各预测类型的准确率:")
for pred_type in prediction_type_stats.index:
    type_results = detailed_results[detailed_results['prediction_type'] == pred_type]
    type_accuracy = type_results['is_correct'].mean()
    print(f"{pred_type}: {type_accuracy:.2%}")

ValueError: 4 columns passed, passed data had 3 columns

In [73]:
import numpy as np
import pandas as pd


def parse_complete_data():
    """解析图片中的完整数据集"""
    # 从图片中提取的所有数据
    data = [
        [0.2234, 0.3562, 0.4204],
        [0.5249, 0.2244, 0.2506],
        [0.4450, 0.2268, 0.3283],
        [0.1884, 0.2577, 0.5539],
        [0.2612, 0.3017, 0.4371],
        [0.6430, 0.1815, 0.1754],
        [0.6069, 0.2125, 0.1806],
        [0.5983, 0.2026, 0.1991],
        [0.2196, 0.1928, 0.5876],
        [0.3428, 0.2075, 0.4497],
        [0.5089, 0.1817, 0.3094],
        [0.3636, 0.2620, 0.3743],
        [0.2373, 0.1926, 0.5701],
        [0.1937, 0.2544, 0.5520],
        [0.4933, 0.2570, 0.2497],
        [0.5702, 0.2070, 0.2228],
        [0.3995, 0.2823, 0.3182]
    ]
    return pd.DataFrame(data, columns=['class0', 'class1', 'class2'])

def evaluate_predictions(df, threshold_single=0.5, threshold_gap=0.15, threshold_gap_2=0.1, threshold_sum=0.79):
    """简化版本的评估预测结果"""
    results = []
    
    # 遍历数据框每一行进行预测
    for idx, row in df.iterrows():
        probs = [row['class0'], row['class1'], row['class2']]
        
        # 获取排序后的概率和对应的类别索引
        sorted_indices = np.argsort(probs)[::-1]
        sorted_probs = np.array(probs)[sorted_indices]
        
        max_prob = sorted_probs[0]
        second_prob = sorted_probs[1]
        last_prob = sorted_probs[-1]
        prob_gap = max_prob - second_prob
        
        # 确定预测策略
        if (max_prob >= 0.35) and (second_prob >= 0.35) and (max_prob <= 0.4) and (second_prob <= 0.4):
            predictions = [1]
            pred_type = "draw 特殊处理1"
            print("draw 特殊处理1: ", probs)
        # elif (max_prob >= 0.35) and (second_prob >= 0.35) and last_prob < 0.2:
        #     predictions = [1]
        #     pred_type = "draw 特殊处理1"
        elif probs[1] >= 0.35:
            predictions = [1]
            pred_type = "draw 特殊处理2"
            print("draw 特殊处理2: ", probs)
        elif second_prob >= 0.28 and second_prob <= 0.32 and np.abs(second_prob - last_prob) <= 0.01:
            predictions = [1]
            pred_type = "draw 特殊处理3"
            print("draw 特殊处理3: ", probs)
        # elif max_prob >= 0.49 and max_prob <= 0.51:
        #     predictions = [1]
        #     pred_type = "draw 特殊处理4"
        elif (second_prob - last_prob) <= 0.01:
            if sorted_indices[1] == 1:
                predictions = [2]
            else:
                predictions = [1]
            pred_type = "home and away 特殊处理1"
            print("home and away特殊处理1: ", probs)
        elif (second_prob - last_prob) <= 0.015 and (second_prob <= 0.23 or sorted_indices[1] == 1):
            if sorted_indices[0] == 0:
                predictions = [1]
            else:
                predictions = [sorted_indices[0]]
            pred_type = "draw 特殊处理4"
            print("draw 特殊处理4: ", probs)
        elif (second_prob - last_prob) <= 0.02:
            predictions = [0]
            pred_type = "away 特殊处理1"
            print("away 特殊处理1: ", probs)
        elif ((second_prob - last_prob) <= 0.1) and (max_prob >= 0.4) and (max_prob <= 0.45):
            predictions = [sorted_indices[0], sorted_indices[1]]
            pred_type = "双重预测 特殊处理1"
            print("双重预测 特殊处理1: ", probs)
        elif (max_prob + second_prob) >= threshold_sum:
            predictions = [sorted_indices[0], sorted_indices[1]]
            pred_type = "双重预测"
            print("双重预测: ", probs)
        elif max_prob >= threshold_single and prob_gap >= threshold_gap:
            predictions = [sorted_indices[0]]
            pred_type = "单一预测"
            print("单一预测: ", probs)
        else:
            predictions = [sorted_indices[0]]
            pred_type = "默认单一预测"
            print("默认单一预测: ", probs)
        
        # 保存预测结果
        results.append({
            'index': idx + 1,
            'predictions': predictions,
            'prediction_type': pred_type,
            'max_prob': max_prob,
            'second_prob': second_prob
        })
    results_df = pd.DataFrame(results)
    return results_df

df = parse_complete_data()
evaluation = evaluate_predictions(df)
print(len(evaluation))
evaluation[['index', 'predictions']]

draw 特殊处理2:  [0.2234, 0.3562, 0.4204]
单一预测:  [0.5249, 0.2244, 0.2506]
默认单一预测:  [0.445, 0.2268, 0.3283]
双重预测:  [0.1884, 0.2577, 0.5539]
双重预测 特殊处理1:  [0.2612, 0.3017, 0.4371]
home and away特殊处理1:  [0.643, 0.1815, 0.1754]
双重预测:  [0.6069, 0.2125, 0.1806]
home and away特殊处理1:  [0.5983, 0.2026, 0.1991]
双重预测:  [0.2196, 0.1928, 0.5876]
双重预测:  [0.3428, 0.2075, 0.4497]
双重预测:  [0.5089, 0.1817, 0.3094]
draw 特殊处理1:  [0.3636, 0.262, 0.3743]
双重预测:  [0.2373, 0.1926, 0.5701]
双重预测:  [0.1937, 0.2544, 0.552]
home and away特殊处理1:  [0.4933, 0.257, 0.2497]
away 特殊处理1:  [0.5702, 0.207, 0.2228]
默认单一预测:  [0.3995, 0.2823, 0.3182]
17


,index,predictions
0,1,[1]
1,2,[0]
2,3,[0]
3,4,"[2, 1]"
4,5,"[2, 1]"
5,6,[2]
6,7,"[0, 1]"
7,8,[2]
8,9,"[2, 0]"
9,10,"[2, 0]"


### 20250223 改良 旧

In [75]:
import numpy as np
import pandas as pd


def parse_complete_data():
    """解析图片中的完整数据集"""
    # 从图片中提取的所有数据
    data = [
        [0.2234, 0.3562, 0.4204],
        [0.5249, 0.2244, 0.2506],
        [0.4450, 0.2268, 0.3283],
        [0.1884, 0.2577, 0.5539],
        [0.2612, 0.3017, 0.4371],
        [0.6430, 0.1815, 0.1754],
        [0.6069, 0.2125, 0.1806],
        [0.5983, 0.2026, 0.1991],
        [0.2196, 0.1928, 0.5876],
        [0.3428, 0.2075, 0.4497],
        [0.5089, 0.1817, 0.3094],
        [0.3636, 0.2620, 0.3743],
        [0.2373, 0.1926, 0.5701],
        [0.1937, 0.2544, 0.5520],
        [0.4933, 0.2570, 0.2497],
        [0.5702, 0.2070, 0.2228],
        [0.3995, 0.2823, 0.3182]
    ]
    return pd.DataFrame(data, columns=['class0', 'class1', 'class2'])

def evaluate_predictions(df, threshold_single=0.5, threshold_gap=0.15, threshold_gap_2=0.1, threshold_sum=0.79):
    """简化版本的评估预测结果"""
    results = []
    
    # 遍历数据框每一行进行预测
    for idx, row in df.iterrows():
        probs = [row['class0'], row['class1'], row['class2']]
        
        # 获取排序后的概率和对应的类别索引
        sorted_indices = np.argsort(probs)[::-1]
        sorted_probs = np.array(probs)[sorted_indices]
        
        max_prob = sorted_probs[0]
        second_prob = sorted_probs[1]
        last_prob = sorted_probs[-1]
        prob_gap = max_prob - second_prob
        
        # 确定预测策略
        if probs[1] >= 0.34:
            predictions = [1]
            pred_type = "draw 特殊处理1"
            print("draw 特殊处理1: ", probs)
        elif (max_prob >= 0.35) and (second_prob >= 0.35) and (max_prob <= 0.4) and (second_prob <= 0.4):
            predictions = [1]
            pred_type = "draw 特殊处理2"
            print("draw 特殊处理2: ", probs)
        elif probs[1] >= 0.245 and probs[1] <= 0.255:
            predictions = [1]
            pred_type = "draw 特殊处理3"
            print("draw 特殊处理3: ", probs)
        elif probs[1] > 0.26:
            if sorted_indices[0] == 0:
                predictions = [2]
            elif sorted_indices[0] == 2:
                predictions = [0]
            pred_type = "反向 特殊处理1"
            print("反向 特殊处理1: ", probs)
        elif ((probs[1] - probs[2]) >= -0.05) and ((probs[1] - probs[2]) <= 0):
            if sorted_indices[0] == 0:
                predictions = [2]
            elif sorted_indices[0] == 2:
                predictions = [0]
            pred_type = "反向 特殊处理2"
            print("反向 特殊处理2: ", probs)
        else:
            predictions = [sorted_indices[0]]
            pred_type = "默认单一预测"
            print("默认单一预测: ", probs)
        if last_prob < 0.18:
            if predictions[0] == 1:
                predictions = sorted_indices[0]
            elif predictions == sorted_indices[0]:
                predictions = [sorted_indices[1], sorted_indices[2]]
            pred_type = "no orig 处理"
            print("no orig 处理: ", probs)
        # 保存预测结果
        results.append({
            'index': idx + 1,
            'predictions': predictions,
            'prediction_type': pred_type,
            'max_prob': max_prob,
            'second_prob': second_prob
        })
    results_df = pd.DataFrame(results)
    return results_df

df = parse_complete_data()
evaluation = evaluate_predictions(df)
print(len(evaluation))
evaluation[['index', 'predictions']]

draw 特殊处理1:  [0.2234, 0.3562, 0.4204]
反向 特殊处理2:  [0.5249, 0.2244, 0.2506]
默认单一预测:  [0.445, 0.2268, 0.3283]
默认单一预测:  [0.1884, 0.2577, 0.5539]
反向 特殊处理1:  [0.2612, 0.3017, 0.4371]
默认单一预测:  [0.643, 0.1815, 0.1754]
no orig 处理:  [0.643, 0.1815, 0.1754]
默认单一预测:  [0.6069, 0.2125, 0.1806]
默认单一预测:  [0.5983, 0.2026, 0.1991]
默认单一预测:  [0.2196, 0.1928, 0.5876]
默认单一预测:  [0.3428, 0.2075, 0.4497]
默认单一预测:  [0.5089, 0.1817, 0.3094]
draw 特殊处理2:  [0.3636, 0.262, 0.3743]
默认单一预测:  [0.2373, 0.1926, 0.5701]
draw 特殊处理3:  [0.1937, 0.2544, 0.552]
默认单一预测:  [0.4933, 0.257, 0.2497]
反向 特殊处理2:  [0.5702, 0.207, 0.2228]
反向 特殊处理1:  [0.3995, 0.2823, 0.3182]
17


,index,predictions
0,1,[1]
1,2,[2]
2,3,[0]
3,4,[2]
4,5,[0]
5,6,"[1, 2]"
6,7,[0]
7,8,[0]
8,9,[2]
9,10,[2]


### 20200223 改良

In [6]:
import numpy as np
import pandas as pd


def parse_complete_data():
    """解析图片中的完整数据集"""
    # 从图片中提取的所有数据
    data = [
[0.2234,0.3562,0.4204],
[0.5249,0.2244,0.2506],
[0.445,0.2268,0.3283],
[0.1884,0.2577,0.5539],
[0.2612,0.3017,0.4371],
[0.643,0.1815,0.1754],
[0.6069,0.2125,0.1806],
[0.5983,0.2026,0.1991],
[0.2196,0.1928,0.5876],
[0.3428,0.2075,0.4497],
[0.5089,0.1817,0.3094],
[0.3636,0.262,0.3743],
[0.2373,0.1926,0.5701],
[0.1937,0.2544,0.552],
[0.4933,0.257,0.2497],
[0.5702,0.207,0.2228],
[0.3995,0.2823,0.3182]

    ]
    return pd.DataFrame(data, columns=['class0', 'class1', 'class2'])

def evaluate_predictions(df, threshold_single=0.5, threshold_gap=0.15, threshold_gap_2=0.1, threshold_sum=0.79):
    """简化版本的评估预测结果"""
    results = []
    
    # 遍历数据框每一行进行预测
    for idx, row in df.iterrows():
        probs = [row['class0'], row['class1'], row['class2']]
        
        # 获取排序后的概率和对应的类别索引
        sorted_indices = np.argsort(probs)[::-1]
        sorted_probs = np.array(probs)[sorted_indices]
        
        max_prob = sorted_probs[0]
        second_prob = sorted_probs[1]
        last_prob = sorted_probs[-1]
        prob_gap = max_prob - second_prob
        
        # 确定预测策略
        if probs[1] >= 0.34:
            predictions = [1]
            pred_type = "draw 特殊处理1"
            print("draw 特殊处理1: ", probs)
        elif (probs[0] > 0.3) and (probs[2] > 0.3) and (probs[0] < 0.4) and (probs[2] < 0.4) and np.abs(probs[0] - probs[2]) <= 0.08 and (probs[1] <= 0.3):
            predictions = [1]
            pred_type = "draw 特殊处理2"
            print("draw 特殊处理2: ", probs)
        elif probs[1] >= 0.245 and probs[1] <= 0.255:
            predictions = [1]
            pred_type = "draw 特殊处理3"
            print("draw 特殊处理3: ", probs)
        # elif probs[1] >= 0.20 and probs[1] <= 0.22 and (second_prob - last_prob) <= 0.006:
        #     predictions = [1]
        #     pred_type = "draw 特殊处理4"
        #     print("draw 特殊处理4: ", probs)
        elif probs[1] > 0.26 and (np.abs(probs[0] - probs[2]) >= 0.28 or np.abs(probs[0] - probs[2]) <= 0.18):
            if sorted_indices[0] == 0:
                predictions = [2]
            elif sorted_indices[0] == 2:
                predictions = [0]
            pred_type = "反向 特殊处理1"
            print("反向 特殊处理1: ", probs)
        elif ((probs[1] - probs[2]) >= -0.04) and ((probs[1] - probs[2]) <= 0):
            if sorted_indices[0] == 0:
                predictions = [2]
            elif sorted_indices[0] == 2:
                predictions = [0]
            pred_type = "反向 特殊处理2"
            print("反向 特殊处理2: ", probs)
        elif (probs[1] >= 0.22) and (probs[1] <= 0.23):
            if sorted_indices[0] == 0:
                predictions = [2]
            elif sorted_indices[0] == 2:
                predictions = [0]
            pred_type = "反向 特殊处理3"
            print("反向 特殊处理3: ", probs)
        else:
            predictions = [sorted_indices[0]]
            pred_type = "默认单一预测"
            print("默认单一预测: ", probs)
        if probs[1] < 0.18:
            predictions = [sorted_indices[0], sorted_indices[1]]
            pred_type = "no orig 处理"
            print("no orig 处理: ", probs)
        # 保存预测结果
        results.append({
            'index': idx + 1,
            'predictions': predictions,
            'prediction_type': pred_type,
            'max_prob': max_prob,
            'second_prob': second_prob
        })
    results_df = pd.DataFrame(results)
    return results_df

df = parse_complete_data()
evaluation = evaluate_predictions(df)
print(len(evaluation))
evaluation[['index', 'predictions']]

draw 特殊处理1:  [0.2234, 0.3562, 0.4204]
反向 特殊处理2:  [0.5249, 0.2244, 0.2506]
反向 特殊处理3:  [0.445, 0.2268, 0.3283]
默认单一预测:  [0.1884, 0.2577, 0.5539]
反向 特殊处理1:  [0.2612, 0.3017, 0.4371]
默认单一预测:  [0.643, 0.1815, 0.1754]
默认单一预测:  [0.6069, 0.2125, 0.1806]
默认单一预测:  [0.5983, 0.2026, 0.1991]
默认单一预测:  [0.2196, 0.1928, 0.5876]
默认单一预测:  [0.3428, 0.2075, 0.4497]
默认单一预测:  [0.5089, 0.1817, 0.3094]
draw 特殊处理2:  [0.3636, 0.262, 0.3743]
默认单一预测:  [0.2373, 0.1926, 0.5701]
draw 特殊处理3:  [0.1937, 0.2544, 0.552]
默认单一预测:  [0.4933, 0.257, 0.2497]
反向 特殊处理2:  [0.5702, 0.207, 0.2228]
反向 特殊处理1:  [0.3995, 0.2823, 0.3182]
17


,index,predictions
0,1,[1]
1,2,[2]
2,3,[2]
3,4,[2]
4,5,[0]
5,6,[0]
6,7,[0]
7,8,[0]
8,9,[2]
9,10,[2]


### 改良 20250225

In [3]:
import numpy as np
import pandas as pd


def parse_complete_data():
    """解析图片中的完整数据集"""
    # 从图片中提取的所有数据
    data = [
        [0.6920, 0.2052, 0.1028],
        [0.5084, 0.2774, 0.2142],
        [0.1735, 0.2418, 0.5847],
        [0.2221, 0.2688, 0.5091],
        [0.2919, 0.2789, 0.4292],
        [0.3989, 0.2900, 0.3112],
        [0.1297, 0.2225, 0.6478],
        [0.3887, 0.2896, 0.3217],
        [0.2649, 0.2944, 0.4408],
        [0.1799, 0.2546, 0.5655],
        [0.3214, 0.2877, 0.3908],
        [0.4448, 0.2692, 0.2860],
        [0.2772, 0.2958, 0.4269],
        [0.4396, 0.2833, 0.2771],
        [0.2855, 0.2888, 0.4257],
        [0.2498, 0.2860, 0.4642],
        [0.4883, 0.2738, 0.2379],
        [0.5279, 0.2699, 0.2022],
        [0.3949, 0.2808, 0.3243],
        [0.1293, 0.2079, 0.6628],
        [0.4183, 0.2742, 0.3075]
    ]
    return pd.DataFrame(data, columns=['class0', 'class1', 'class2'])

def evaluate_predictions(df, threshold_single=0.5, threshold_gap=0.15, threshold_gap_2=0.1, threshold_sum=0.79):
    """简化版本的评估预测结果"""
    results = []
    
    # 遍历数据框每一行进行预测
    for idx, row in df.iterrows():
        probs = [row['class0'], row['class1'], row['class2']]
        
        # 获取排序后的概率和对应的类别索引
        sorted_indices = np.argsort(probs)[::-1]
        sorted_probs = np.array(probs)[sorted_indices]
        
        max_prob = sorted_probs[0]
        second_prob = sorted_probs[1]
        last_prob = sorted_probs[-1]
        prob_gap = max_prob - second_prob
        
        # 确定预测策略
        if probs[1] >= 0.34:
            predictions = [1]
            pred_type = "draw 特殊处理1"
            print("draw 特殊处理1: ", probs)
        elif probs[1] >= 0.295:
            predictions = [1]
            pred_type = "draw 特殊处理4"
            print("draw 特殊处理1: ", probs)
        elif (probs[0] > 0.3) and (probs[2] > 0.3) and (probs[0] < 0.4) and (probs[2] < 0.4) and np.abs(probs[0] - probs[2]) <= 0.08 and (probs[1] <= 0.3):
            predictions = [1]
            pred_type = "draw 特殊处理2"
            print("draw 特殊处理2: ", probs)
        elif probs[1] >= 0.245 and probs[1] <= 0.255:
            predictions = [1]
            pred_type = "draw 特殊处理3"
            print("draw 特殊处理3: ", probs)
        elif probs[2] >= 0.4 and probs[2] <= 0.5:
            predictions = [0, 1]
            pred_type = "home 特殊处理1"
            print("home 特殊处理1: ", probs)           
        # elif probs[1] >= 0.20 and probs[1] <= 0.22 and (second_prob - last_prob) <= 0.006:
        #     predictions = [1]
        #     pred_type = "draw 特殊处理4"
        #     print("draw 特殊处理4: ", probs)
        # elif probs[1] > 0.26 and (np.abs(probs[0] - probs[2]) >= 0.28 or np.abs(probs[0] - probs[2]) <= 0.18):
        #     if sorted_indices[0] == 0:
        #         predictions = [2]
        #     elif sorted_indices[0] == 2:
        #         predictions = [0]
        #     pred_type = "反向 特殊处理1"
        #     print("反向 特殊处理1: ", probs)
        elif ((probs[1] - probs[2]) >= -0.035) and ((probs[1] - probs[2]) <= 0):
            if sorted_indices[0] == 0:
                predictions = [2]
            elif sorted_indices[0] == 2:
                predictions = [0]
            pred_type = "反向 特殊处理2"
            print("反向 特殊处理2: ", probs)
        else:
            predictions = [sorted_indices[0]]
            pred_type = "默认单一预测"
            print("默认单一预测: ", probs)
        if probs[1] < 0.18:
            predictions = [sorted_indices[0], sorted_indices[1]]
            pred_type = "no orig 处理"
            print("no orig 处理: ", probs)
        # 保存预测结果
        results.append({
            'index': idx + 1,
            'predictions': predictions,
            'prediction_type': pred_type,
            'max_prob': max_prob,
            'second_prob': second_prob
        })
    results_df = pd.DataFrame(results)
    return results_df

df = parse_complete_data()
evaluation = evaluate_predictions(df)
print(len(evaluation))
evaluation[['index', 'predictions']]

默认单一预测:  [0.692, 0.2052, 0.1028]
默认单一预测:  [0.5084, 0.2774, 0.2142]
默认单一预测:  [0.1735, 0.2418, 0.5847]
默认单一预测:  [0.2221, 0.2688, 0.5091]
home 特殊处理1:  [0.2919, 0.2789, 0.4292]
反向 特殊处理2:  [0.3989, 0.29, 0.3112]
默认单一预测:  [0.1297, 0.2225, 0.6478]
draw 特殊处理2:  [0.3887, 0.2896, 0.3217]
home 特殊处理1:  [0.2649, 0.2944, 0.4408]
draw 特殊处理3:  [0.1799, 0.2546, 0.5655]
draw 特殊处理2:  [0.3214, 0.2877, 0.3908]
反向 特殊处理2:  [0.4448, 0.2692, 0.286]
draw 特殊处理1:  [0.2772, 0.2958, 0.4269]
默认单一预测:  [0.4396, 0.2833, 0.2771]
home 特殊处理1:  [0.2855, 0.2888, 0.4257]
home 特殊处理1:  [0.2498, 0.286, 0.4642]
默认单一预测:  [0.4883, 0.2738, 0.2379]
默认单一预测:  [0.5279, 0.2699, 0.2022]
draw 特殊处理2:  [0.3949, 0.2808, 0.3243]
默认单一预测:  [0.1293, 0.2079, 0.6628]
反向 特殊处理2:  [0.4183, 0.2742, 0.3075]
21


,index,predictions
0,1,[0]
1,2,[0]
2,3,[2]
3,4,[2]
4,5,"[0, 1]"
5,6,[2]
6,7,[2]
7,8,[1]
8,9,"[0, 1]"
9,10,[1]
